In [19]:
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import numpy as np
import scipy.io as sio
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import tree
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
import random
from collections import Counter
np.random.seed(0)
random.seed(0)
ran=0
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn import preprocessing

from sklearn.svm import SVC

from sklearn.decomposition import PCA

from sklearn.neural_network import MLPClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope



import seaborn as sn
import pandas as pd
from sklearn.utils import class_weight

FolderSave=r"C:\Users\zz587-admin\Documents\sleep center paper\figure\python\comp"

In [20]:
def ImportData(Data_ver,opt):
    Folder=r"C:\Users\zz587-admin\Documents\sleep center\CNN_algorithm\Output"
    
    # #Data_ver=r'\CNNFeat_pred_v4'
    # Data_ver=r'\Feat_pred_v4'
    Folder=Folder + Data_ver 
    print(Data_ver)
    path=Folder+r'\featureAll'+Data_ver+"_feature_data.mat"

    data = sio.loadmat(path)
    featureNCS = data['featureNCS']
    featureNCS=np.nan_to_num(featureNCS, nan=0,posinf=1000)
    # featureNCS=np.delete(featureNCS,np.arange(16,37),1)
    featureNCS_wo = featureNCS[:,0:-4]
    labelNCS = data['labelNCS']
    labelNCS_2 = data['labelNCS_2']
    labelNCS_3 = data['labelNCS_3']
    labelNCS_4 = data['labelNCS_4']

    featurePSG= data['featurePSG2']
    featurePSG=np.nan_to_num(featurePSG, nan=0,posinf=1000)
    featurePSG_wo = featurePSG[:,0:-4]
    labelPSG = data['labelPSG2']
    labelPSG_2 = data['labelPSG2_2']
    labelPSG_3 = data['labelPSG2_3']
    labelPSG_4 = data['labelPSG2_4']
    caseNumNCS=data['caseNumNCS']
    # caseNumPSG=data['caseNumPSG']
    caseNumPSG=data['caseNumPSG2']

    featureNCS_s=np.delete(featureNCS,np.arange(16,37),1)
    featureNCS_wo_s = featureNCS_s[:,0:-4]
    featurePSG_s=np.delete(featurePSG,np.arange(16,37),1)
    featurePSG_wo_s = featurePSG_s[:,0:-4] 
    if opt=='short':
        featureNCS=featureNCS_s
        featureNCS_wo=featureNCS_wo_s
        featurePSG=featurePSG_s
        featurePSG_wo=featurePSG_wo_s

    return featureNCS,featureNCS_wo,labelNCS,labelNCS_3,caseNumNCS,featurePSG,featurePSG_wo,labelPSG,labelPSG_3,caseNumPSG

In [21]:


#function for k-fold CV. 
# return  y  =(truth,predict),  acc_all = all accuracy of all folds  ,acc =  mean accuracy ,cm = all data's confusion matrix ,featureName2,importance2 =average feat imp


#opt[0]  = 0 use balanced sample weight   opt[0]  = 0  specify weight
#opt[1]  = b max_depth
#opt[2]  = num n_estimators
def k_foldCV(x,y,opt,w):
    rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=1,random_state=0)
    b=int(opt[1])
    num=int(opt[2])
    y=y.ravel()
    # x_ = preprocessing.StandardScaler().fit_transform(x)
    # x=x_
    y_p_all=[]
    y_test_all=[]
    acc_all=[]
    impt=[]
    
    bal='balanced'
    if x.shape[1]==20 or x.shape[1]==16:
        featureName=['mean(br)', 'std(br)', 'mean(pp)' ,'std(pp)' ,'mean(in)' ,'std(in)' ,'mean(ex)' ,'std(ex)',\
                'skew_mean' , 'kurt_mean' ,'entro',\
                'per_power', 'cycle', 'covBR' ,'covPP', 'void_t',\
                    'mean(spo2)' ,'std(spo2)', 'percent(spo2)', 'min(spo2)']
    else:
        featureName=['mean(br)', 'std(br)', 'mean(pp)' ,'std(pp)' ,'mean(in)' ,'std(in)' ,'mean(ex)' ,'std(ex)',\
                    'skew_mean' , 'kurt_mean' ,'entro',\
                    'per_power', 'cycle', 'covBR' ,'covPP', 'void_t',\
                        'max(in)','max(ex)','max(br)','min(br)','min(pp)',\
                        'Cor(br)','SD(br)','Cor(pp)','SD(pp)','Cor(in)','SD(in)','Cor(ex)','SD(ex)',\
                        'psd(1)','psd(2)','psd(3)','psd(4)','fmax(1)','fmax(2)','fmax(3)','fmax(4)',\
                        'mean(spo2)' ,'std(spo2)', 'percent(spo2)', 'min(spo2)']
    
    for train_ind, test_ind in rkf.split(x,y):
        
        X_train=x[train_ind] 
        X_test=x[test_ind]
        y_train=y[train_ind] 
        y_test=y[test_ind]
        
        
        weight={0:1,1:w}
        #balance the sample 
        if opt[0]==1:
            s=class_weight.compute_sample_weight(weight,y_train)         
            clf = RandomForestClassifier(max_depth=b,n_estimators=num,max_features=None,random_state=0,class_weight=weight).fit(X_train,y_train,sample_weight=s)
            
        if opt[0]==0:            
            clf = RandomForestClassifier(max_depth=b,n_estimators=num,max_features=None,random_state=0,class_weight="balanced").fit(X_train,y_train)
        #clf = SVC(C=3, kernel="rbf").fit(X_train,y_train)
        y_p = clf.predict(X_test)
        acc=accuracy_score(y_test, y_p)
        
        
        for i in range(len(y_p)):
            y_p_all.append(y_p[i])
            y_test_all.append(y_test[i])
        
        acc_all.append(acc)

        impt_temp=clf.feature_importances_
        impt.append(impt_temp)
    #clf = RandomForestClassifier(max_depth=5,n_estimators=50,max_features=None,random_state=ran).fit(X_train,y_train)
    # clf = KNeighborsClassifier(n_neighbors=10).fit(X_train,y_train)


    impt=np.array(impt) 
    importance2=np.mean(impt,axis=0)
    seq=np.argsort(importance2)
    featureName=np.array(featureName)
    featureName2=featureName[seq]
    importance2=importance2[seq]
    importance2=np.flip(importance2)
    featureName2=np.flip(featureName2)


    y_test_all=np.array(y_test_all)
    y_p_all=np.array(y_p_all)
    y=np.stack((y_test_all, y_p_all))

    cm=confusion_matrix(y_test_all, y_p_all)

    acc=accuracy_score(y_test_all, y_p_all)
    print(acc)
    print(cm)
    sen=[0,0,0]
    sen[0]=acc
    if len(cm)==2:
        sen[1]=cm[1,1]/(cm[1,1]+cm[1,0])
        sen[2]=cm[0,0]/(cm[0,0]+cm[0,1])
        
        print('acc=' + str(round(sen[0], 3)),'sen=' + str(round(sen[1], 3)) +  ' spe=' + str(round(sen[2], 3)))  
    else:  
        print(acc)

    return y,acc_all,acc,cm,featureName2,importance2,sen


#function for subject-independent CV.  
#input: x=features y=labels  group= case group 
# return  y  =(truth,predict),  acc_all = all accuracy for all cases  ,acc =  mean accuracy ,cm = all data's confusion matrix ,featureName2,importance2 =average feat imp

def groupCV(x,y,group,opt,w):
    a=5
    b=10
    num=50
    rkf = LeaveOneGroupOut()
    if x.shape[1]==20 or x.shape[1]==16:
        featureName=['mean(br)', 'std(br)', 'mean(pp)' ,'std(pp)' ,'mean(in)' ,'std(in)' ,'mean(ex)' ,'std(ex)',\
                'skew_mean' , 'kurt_mean' ,'entro',\
                'per_power', 'cycle', 'covBR' ,'covPP', 'void_t',\
                    'mean(spo2)' ,'std(spo2)', 'percent(spo2)', 'min(spo2)']
    else:
        featureName=['mean(br)', 'std(br)', 'mean(pp)' ,'std(pp)' ,'mean(in)' ,'std(in)' ,'mean(ex)' ,'std(ex)',\
                    'skew_mean' , 'kurt_mean' ,'entro',\
                    'per_power', 'cycle', 'covBR' ,'covPP', 'void_t',\
                        'max(in)','max(ex)','max(br)','min(br)','min(pp)',\
                        'Cor(br)','SD(br)','Cor(pp)','SD(pp)','Cor(in)','SD(in)','Cor(ex)','SD(ex)',\
                        'psd(1)','psd(2)','psd(3)','psd(4)','fmax(1)','fmax(2)','fmax(3)','fmax(4)',\
                        'mean(spo2)' ,'std(spo2)', 'percent(spo2)', 'min(spo2)']
    # x_ = preprocessing.StandardScaler().fit_transform(x)
    # x=x_
    y_p_all=[]
    y_test_all=[]
    acc_all=[]
    impt=[]
    y=y.ravel()
    group=group.ravel()
   
    
    for train_ind, test_ind in rkf.split(x,y,group):
        
        X_train=x[train_ind] 
        X_test=x[test_ind]
        y_train=y[train_ind] 
        y_test=y[test_ind]
        
                    
        if opt[0]==1:
            s=class_weight.compute_sample_weight(w,y_train)            
            clf = RandomForestClassifier(max_depth=b,n_estimators=num,max_features=None,random_state=0,class_weight=w).fit(X_train,y_train,sample_weight=s)
           
        if opt[0]==0:            
            clf = RandomForestClassifier(max_depth=b,n_estimators=num,max_features=None,random_state=0).fit(X_train,y_train)
        
        y_p = clf.predict(X_test)
        acc=accuracy_score(y_test, y_p)
        
        
        for i in range(len(y_p)):
            y_p_all.append(y_p[i])
            y_test_all.append(y_test[i])
        
        acc_all.append(acc)
        

        impt_temp=clf.feature_importances_
        impt.append(impt_temp)



    impt=np.array(impt) 
    importance2=np.mean(impt,axis=0)
    seq=np.argsort(importance2)
    featureName=np.array(featureName)
    featureName2=featureName[seq]
    importance2=importance2[seq]
    importance2=np.flip(importance2)
    featureName2=np.flip(featureName2)

    y_test_all=np.array(y_test_all)
    y_p_all=np.array(y_p_all)
    y=np.stack((y_test_all, y_p_all))

    cm=confusion_matrix(y_test_all, y_p_all)

    acc=accuracy_score(y_test_all, y_p_all)
    sen=0
    spe=0
   
    if len(cm)==2:
        sen=cm[1,1]/(cm[1,1]+cm[1,0])
        spe=cm[0,0]/(cm[0,0]+cm[0,1])
        
        print('acc=' + str(round(sen[0], 3)),'sen=' + str(round(sen[1], 3)) +  ' spe=' + str(round(sen[2], 3)))  
    else:  
        print(acc)

    return y,acc_all,acc,cm,featureName2,importance2,sen,spe



# get sensitivity and precision from confusion matrix 
def cm_parameter(cm):
    if len(cm)==2:
        sen=cm[1,1]/(cm[1,1]+cm[1,0])
        spe=cm[0,0]/(cm[0,0]+cm[0,1])
        
        
    print('acc=' + str(round(sen[0], 3)),'sen=' + str(round(sen[1], 3)) +  ' spe=' + str(round(sen[2], 3)))    
    return sen,spe


In [22]:
# detect outlier in normal epoch & disorder epoch. 
def Del_outlier(featureNCS,featureNCS_wo,labelNCS,labelNCS_3,caseNumNCS,opt0,opt1,a):
    a0=a[0]   # outlier rate for normal epoch
    a1=a[1]   # outlier rate for disorder epoch

    idx_0=np.where( labelNCS_3 == 0 )[0]  
    caseNumNCS_0=caseNumNCS[idx_0]
    featureNCS_0=featureNCS[idx_0,:]
    featureNCS_wo_0=featureNCS_wo[idx_0,:]
    labelNCS_3_0 = labelNCS_3[idx_0]
    labelNCS_0 = labelNCS[idx_0]




    idx_1=np.where( labelNCS_3 == 1 )[0]  
    caseNumNCS_1=caseNumNCS[idx_1]
    featureNCS_1=featureNCS[idx_1,:]
    featureNCS_wo_1=featureNCS_wo[idx_1,:]
    labelNCS_3_1 = labelNCS_3[idx_1] 
    labelNCS_1 = labelNCS[idx_1] 

    # different outlier methods (4)
    # clf=OneClassSVM(nu=0.25)
    # clf=EllipticEnvelope(contamination=0.1)
    # clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)
    if opt0=='forest':
        clf0 =IsolationForest(contamination=a0,random_state=0)
    if opt0=='local':
        clf0 =LocalOutlierFactor(n_neighbors=50, contamination=a0)
    if opt0=='SVM':
        clf0 =OneClassSVM(nu=a0)
    if opt0=='covar':
        clf0 =EllipticEnvelope(contamination=a0)
 

 

    lier_0 = clf0.fit_predict(featureNCS_wo_0)
    # 1 inlier, -1 outlier

    #delete some bad cases in 0 label    idx_0_del: after delete outliers, remaining index 
    idx_0_del=np.where( lier_0 == 1 )[0]  
    caseNumNCS_0_del=caseNumNCS_0[idx_0_del]
    featureNCS_0_del=featureNCS_0[idx_0_del,:]
    featureNCS_wo_0_del=featureNCS_wo_0[idx_0_del,:]
    labelNCS_3_0_del = labelNCS_3_0[idx_0_del]
    labelNCS_0_del = labelNCS_0[idx_0_del]

    if a1!=0:
        if opt1=='forest':
            clf1 =IsolationForest(contamination=a1,random_state=0)
        if opt1=='local':
            clf1 =LocalOutlierFactor(n_neighbors=50, contamination=a1)
        if opt1=='SVM':
            clf1 =OneClassSVM(nu=a1)
        if opt1=='covar':
            clf1 =EllipticEnvelope(contamination=a1)
        lier_1 = clf1.fit_predict(featureNCS_wo_1)
        # 1 inlier, -1 outlier

        #delete some bad cases in 1 label 
        idx_1_del=np.where( lier_1 == 1 )[0]

    # if not delete outlier in disorder 
    if a1==0:
        idx_1_del=np.arange(0,len(caseNumNCS_1))

    caseNumNCS_1_del=caseNumNCS_1[idx_1_del]
    featureNCS_1_del=featureNCS_1[idx_1_del,:]
    featureNCS_wo_1_del=featureNCS_wo_1[idx_1_del,:]
    labelNCS_3_1_del = labelNCS_3_1[idx_1_del]
    labelNCS_1_del = labelNCS_1[idx_1_del]


    featureNCS_del=np.vstack((featureNCS_0_del,featureNCS_1_del))
    featureNCS_wo_del=np.vstack((featureNCS_wo_0_del,featureNCS_wo_1_del))
    caseNumNCS_del=np.vstack((caseNumNCS_0_del,caseNumNCS_1_del))
    labelNCS_3_del=np.vstack((labelNCS_3_0_del,labelNCS_3_1_del))
    labelNCS_del=np.vstack((labelNCS_0_del,labelNCS_1_del))


    return featureNCS_del,featureNCS_wo_del,caseNumNCS_del,labelNCS_3_del,labelNCS_del


In [23]:
# import comparison data 
opt='short'
Data_ver1=r'\Feat_pred_v3'
Data_ver2=r'\Feat_pred_v2'
Data_ver3=r'\Feat_pred_v1'
Data_ver4=r'\Feat_pred_v4'
Data_ver=[Data_ver1,Data_ver2,Data_ver3,Data_ver4]
# featureNCS_1,featureNCS_wo_1,labelNCS_1,labelNCS_3_1,caseNumNCS_1,featurePSG_1,featurePSG_wo_1,labelPSG_1,labelPSG_3_1,caseNumPSG_1=ImportData(Data_ver1,opt)
# featureNCS_2,featureNCS_wo_2,labelNCS_2,labelNCS_3_2,caseNumNCS_2,featurePSG_2,featurePSG_wo_2,labelPSG_2,labelPSG_3_2,caseNumPSG_2=ImportData(Data_ver2,opt)
# featureNCS_3,featureNCS_wo_3,labelNCS_3,labelNCS_3_3,caseNumNCS_3,featurePSG_3,featurePSG_wo_3,labelPSG_3,labelPSG_3_3,caseNumPSG_3=ImportData(Data_ver3,opt)
# featureNCS_4,featureNCS_wo_4,labelNCS_4,labelNCS_3_4,caseNumNCS_4,featurePSG_4,featurePSG_wo_4,labelPSG_4,labelPSG_3_4,caseNumPSG_4=ImportData(Data_ver4,opt)

In [24]:
## delete some data with bad performance  test over deleted data 
# long features for prediction
#opt[0]  = 0 use balanced sample weight   opt[0]  = 0  specify weight
#opt[1]  = b max_depth
#opt[2]  = num n_estimators
opt=np.zeros([3,4])
w=np.zeros([4,4])
opt[:,0]=[1,20,50]
opt[:,1]=[1,20,50]
opt[:,2]=[1,10,30]
opt[:,3]=[1,20,50]
w[:,0]=[9,8.5,10,10]
w[:,1]=[7,7,8,8]
w[:,2]=[5,5,5,5]
w[:,3]=[4,4,4,4]
w2={0:1,1:5}

r=np.array([0.2,0])
cm_NCS=[]
sen_NCS=[]
cm_NCS_wo=[]
sen_NCS_wo=[]
cm_PSG=[]
sen_PSG=[]
cm_PSG_wo=[]
sen_PSG_wo=[]
for i in range(4):

    featureNCS_1,featureNCS_wo_1,labelNCS_1,labelNCS_3_1,caseNumNCS_1,featurePSG_1,featurePSG_wo_1,labelPSG_1,labelPSG_3_1,caseNumPSG_1=ImportData(Data_ver[i],'long')
    featureNCS_del1,featureNCS_wo_del1,caseNumNCS_del1,labelNCS_3_del1,labelNCS_del1=\
        Del_outlier(featureNCS_1,featureNCS_wo_1,labelNCS_1,labelNCS_3_1,caseNumNCS_1,'forest','forest',r)
    featurePSG_del1,featurePSG_wo_del1,caseNumPSG_del1,labelPSG_3_del1,labelPSG_del1=\
        Del_outlier(featurePSG_1,featurePSG_wo_1,labelPSG_1,labelPSG_3_1,caseNumPSG_1,'forest','forest',r)
    
    opt1=opt[:,i].ravel()
    _, _, _ ,cm_NCS_del,_,_,sen_NCS_del=k_foldCV(featureNCS_del1,labelNCS_3_del1,opt1,w[0,i])
    _, _, _ ,cm_NCS_wo_del,_,_,sen_NCS_wo_del=k_foldCV(featureNCS_wo_del1,labelNCS_3_del1,opt1,w[1,i])
    _, _, _ ,cm_PSG_del,_,_,sen_PSG_del=k_foldCV(featurePSG_del1,labelPSG_3_del1,opt1,w[2,i])
    _, _, _ ,cm_PSG_wo_del,_,_,sen_PSG_wo_del=k_foldCV(featurePSG_wo_del1,labelPSG_3_del1,opt1,w[3,i])
    cm_NCS.append(cm_NCS_del)
    sen_NCS.append(sen_NCS_del)
    cm_NCS_wo.append(cm_NCS_wo_del)
    sen_NCS_wo.append(sen_NCS_wo_del)
    cm_PSG.append(cm_PSG_del)
    sen_PSG.append(sen_PSG_del)
    cm_PSG_wo.append(cm_PSG_wo_del)
    sen_PSG_wo.append(sen_PSG_wo_del)



\Feat_pred_v3
\Feat_pred_v3
0.8677188524041999
[[10526  1482]
 [  496  2449]]
acc=0.868 sen=0.832 spe=0.877
0.860964354978934
[[10510  1498]
 [  581  2364]]
acc=0.861 sen=0.803 spe=0.875
0.770420099684671
[[6822 1829]
 [ 428  752]]
acc=0.77 sen=0.637 spe=0.789
0.6441867561794324
[[5519 3132]
 [ 366  814]]
acc=0.644 sen=0.69 spe=0.638
\Feat_pred_v2
\Feat_pred_v2
0.8379616039452827
[[9885 2027]
 [ 733 4388]]
acc=0.838 sen=0.857 spe=0.83
0.7008160629366523
[[7481 4431]
 [ 665 4456]]
acc=0.701 sen=0.87 spe=0.628
0.6714245202760185
[[5543 2932]
 [ 544 1560]]
acc=0.671 sen=0.741 spe=0.654
0.5683902070138954
[[4350 4125]
 [ 441 1663]]
acc=0.568 sen=0.79 spe=0.513
\Feat_pred_v1
\Feat_pred_v1
0.6532758022119725
[[5428 6645]
 [ 346 7744]]
acc=0.653 sen=0.957 spe=0.45


KeyboardInterrupt: 

In [ ]:
print(w[2,0])

In [ ]:
# group CV  subject -independent 

opt1=[0]
opt2=[1]
w1={0:1,1:2.5}
w2={0:1,1:2.5}

# class weight. balanced . opt1= [0] 

y_NCS_del1_g,acc_all_NCS_del1_g,acc_NCS_del1_g,cm_NCS_del1_g,feat_NCS_del1_g,impt_NCS_del1_g=\
    groupCV(featureNCS_del1,labelNCS_3_del1,caseNumNCS_del1,opt1,w2)
y_NCS_wo_del1_g,acc_all_NCS_wo_del1_g,acc_NCS_wo_del1_g,cm_NCS_wo_del1_g,feat_NCS_wo_del1_g,impt_NCS_wo_del1_g=\
    groupCV(featureNCS_wo_del1,labelNCS_3_del1,caseNumNCS_del1,opt1,w2)


In [ ]:
y_PSG_del1_g,acc_all_PSG_del1_g,acc_PSG_del1_g,cm_PSG_del1_g,feat_PSG_del1_g,impt_PSG_del1_g=\
    groupCV(featurePSG_del1,labelPSG_3_del1,caseNumPSG_del1,opt2,w2)
y_PSG_wo_del1_g,acc_all_PSG_wo_del1_g,acc_PSG_wo_del1_g,cm_PSG_wo_del1_g,feat_PSG_wo_del1_g,impt_PSG_wo_del1_g=\
    groupCV(featurePSG_wo_del1,labelPSG_3_del1,caseNumPSG_del1,opt2,w2)

In [ ]:
# 7 class result 
# class weight. balanced . opt1= [0] 
y_NCS_del1_fc,acc_all_NCS_del1_fc,acc_NCS_del1_fc,cm_NCS_del1_fc,feat_NCS_del1_fc,impt_NCS_del1_fc=k_foldCV(featureNCS_del1,labelNCS_del1,opt1,w2)
y_NCS_wo_del1_fc,acc_all_NCS_wo_del1_fc,acc_NCS_wo_del1_fc,cm_NCS_wo_del1_fc,feat_NCS_wo_del1_fc,impt_NCS_wo_del1_fc=k_foldCV(featureNCS_wo_del1,labelNCS_del1,opt1,w2)


y_PSG_del1_fc,acc_all_PSG_del1_fc,acc_PSG_del1_fc,cm_PSG_del1_fc,feat_PSG_del1_fc,impt_PSG_del1_fc=k_foldCV(featurePSG_del1,labelPSG_del1,opt1,w2)
y_PSG_wo_del1_fc,acc_all_PSG_wo_del1_fc,acc_PSG_wo_del1_fc,cm_PSG_wo_del1_fc,feat_PSG_wo_del1_fc,impt_PSG_wo_del1_fc=k_foldCV(featurePSG_wo_del1,labelPSG_del1,opt1,w2)

In [ ]:
def plotfigFull(cm,acc,feat,impt,CVmethod,tle):

    labelNum=str(3)  #labelling method 1,2,3,4
    size=18
    name1=["True 0", "True 1"]
    name2=["Pred 0", "Pred 1"]
    w=12 #fig size 1      change when label num change
    h=8   #fig size 2
    a=2   #txt position 1   change when label num change
    b=1   #txt position 2

    sen,spe=cm_parameter(cm)
    df_cm = pd.DataFrame(cm, index=name1, columns=name2)
    plt.rcParams["font.family"] = "Times New Roman"
    plt.figure()
    sn.set(font_scale=1.5)
    sn.heatmap(df_cm, annot=True,fmt="d",square=True,cbar=False,annot_kws={"size": size})
    s01='accuracy={n:.3f}\nsensitivity={n11:.3f}\nspecificity={n12:.3f}\nFeature Importance:\n{n1}={s1:.3f}\n{n2}={s2:.3f}\n{n3}={s3:.3f}\n{n4}={s4:.3f}'.\
        format(n=acc,n11=sen,n12=spe,\
        n1=feat[0],s1=impt[0],n2=feat[1],s2=impt[1],n3=feat[2],s3=impt[2],n4=feat[3],s4=impt[3])
    plt.text(a,b,s01,fontsize=size) 
    
    tleSave=CVmethod+tle+ labelNum
    plt.title(tleSave)

    figure = plt.gcf()
    figure.set_size_inches(w, h)
    FolderSave=r"C:/Users/zz587-admin/Documents/sleep center paper/figure/python/cm/"
    plt.savefig(FolderSave+'fullVer'+tleSave+r".png",dpi=300)
    #plt.savefig(FolderSave+tleSave+r".svg")

In [ ]:
def plotfig(cm,acc,feat,impt,CVmethod,tle):

    labelNum=str(3)  #labelling method 1,2,3,4
    size=18
    name1=["True 0", "True 1"]
    name2=["Pred 0", "Pred 1"]
    w=7 #fig size 1      change when label num change
    h=6   #fig size 2
    a=2   #txt position 1   change when label num change
    b=1   #txt position 2

    sen,pre=cm_parameter(cm)
    df_cm = pd.DataFrame(cm, index=name1, columns=name2)
    plt.rcParams["font.family"] = "Times New Roman"
    plt.figure()
    sn.set(font_scale=1.5)
    sn.heatmap(df_cm, annot=True,fmt="d",square=True,cbar=False,annot_kws={"size": size})

    
    tleSave=CVmethod+tle+ labelNum
    plt.title(tle)

    figure = plt.gcf()
    figure.set_size_inches(w, h)
    FolderSave=r"C:/Users/zz587-admin/Documents/sleep center paper/figure/python/cm/"
    plt.savefig(FolderSave+tleSave+r".png",dpi=300)
    #plt.savefig(FolderSave+tleSave+r".svg")


CVmethod='long features Weighted class1_4  k_fold Outlier Removed(0.2) Isolation Forest'
tle1='NCS + SpO2  Prediction'
plotfig(cm_NCS_del1,acc_NCS_del1,feat_NCS_del1,impt_NCS_del1,CVmethod,tle1)
tle2='NCS   Prediction'
plotfig(cm_NCS_wo_del1,acc_NCS_wo_del1,feat_NCS_wo_del1,impt_NCS_wo_del1,CVmethod,tle2)
tle3='PSG + SpO2  Prediction'
plotfig(cm_PSG_del1,acc_PSG_del1,feat_PSG_del1,impt_PSG_del1,CVmethod,tle3)
tle4='PSG  Prediction'
plotfig(cm_PSG_wo_del1,acc_PSG_wo_del1,feat_PSG_wo_del1,impt_PSG_wo_del1,CVmethod,tle4)

In [ ]:
#CVmethod='long features Weighted class1_2.5  group Outlier Removed(0.2) Isolation Forest'
plotfigFull(cm_NCS_del1,acc_NCS_del1,feat_NCS_del1,impt_NCS_del1,CVmethod,tle1)
plotfigFull(cm_NCS_wo_del1,acc_NCS_wo_del1,feat_NCS_wo_del1,impt_NCS_wo_del1,CVmethod,tle2)
plotfigFull(cm_PSG_del1,acc_PSG_del1,feat_PSG_del1,impt_PSG_del1,CVmethod,tle3)
plotfigFull(cm_PSG_wo_del1,acc_PSG_wo_del1,feat_PSG_wo_del1,impt_PSG_wo_del1,CVmethod,tle4)

CVmethod2='long features Weighted class1_2.5  group Outlier Removed(0.2) Isolation Forest'
plotfig(cm_NCS_del1_g,acc_NCS_del1_g,feat_NCS_del1_g,impt_NCS_del1_g,CVmethod2,tle1)
plotfig(cm_NCS_wo_del1_g,acc_NCS_wo_del1_g,feat_NCS_wo_del1_g,impt_NCS_wo_del1_g,CVmethod2,tle2)
plotfig(cm_PSG_del1_g,acc_PSG_del1_g,feat_PSG_del1_g,impt_PSG_del1_g,CVmethod2,tle3)
plotfig(cm_PSG_wo_del1_g,acc_PSG_wo_del1_g,feat_PSG_wo_del1_g,impt_PSG_wo_del1_g,CVmethod2,tle4)

plotfigFull(cm_NCS_del1_g,acc_NCS_del1_g,feat_NCS_del1_g,impt_NCS_del1_g,CVmethod2,tle1)
plotfigFull(cm_NCS_wo_del1_g,acc_NCS_wo_del1_g,feat_NCS_wo_del1_g,impt_NCS_wo_del1_g,CVmethod2,tle2)
plotfigFull(cm_PSG_del1_g,acc_PSG_del1_g,feat_PSG_del1_g,impt_PSG_del1_g,CVmethod2,tle3)
plotfigFull(cm_PSG_wo_del1_g,acc_PSG_wo_del1_g,feat_PSG_wo_del1_g,impt_PSG_wo_del1_g,CVmethod2,tle4)

In [ ]:
def plotfigFullClass(cm,acc,feat,impt,CVmethod,tle):

    labelNum=str(1)  #labelling method 1,2,3,4
    size=18
    name1=["0", "S","A","H","OSA","MA","CSA"]
    name2=name1
    w=15 #fig size 1      change when label num change
    h=13   #fig size 2
    

    
    df_cm = pd.DataFrame(cm, index=name1, columns=name2)
    plt.rcParams["font.family"] = "Times New Roman"
    plt.figure()
    sn.set(font_scale=2)
    sn.heatmap(df_cm, annot=True,fmt="d",square=True,cbar=False,annot_kws={"size": size})

    
    tleSave=tle+CVmethod+ labelNum
    plt.title(tle)

    figure = plt.gcf()
    figure.set_size_inches(w, h)
    FolderSave=r"C:/Users/zz587-admin/Documents/sleep center paper/figure/python/cm/"
    plt.savefig(FolderSave+tleSave+r".png",dpi=300)


CVmethod3=' k-fold '
plotfigFullClass(cm_NCS_del1_fc,acc_NCS_del1_fc,feat_NCS_del1_fc,impt_NCS_del1_fc,CVmethod3,tle1)
plotfigFullClass(cm_NCS_wo_del1_fc,acc_NCS_wo_del1_fc,feat_NCS_wo_del1_fc,impt_NCS_wo_del1_fc,CVmethod3,tle2)
plotfigFullClass(cm_PSG_del1_fc,acc_PSG_del1_fc,feat_PSG_del1_fc,impt_PSG_del1_fc,CVmethod3,tle3)
plotfigFullClass(cm_PSG_wo_del1_fc,acc_PSG_wo_del1_fc,feat_PSG_wo_del1_fc,impt_PSG_wo_del1_fc,CVmethod3,tle4)



In [ ]:
CVmethod='Weighted class1_6  kfold'
labelNum=str(3)  #labelling method 1,2,3,4
size=18
name1=["True 0", "True Apnea"]
name2=["Pred 0", "Pred apnea"]
w=12 #fig size 1      change when label num change
h=8   #fig size 2
a=2   #txt position 1   change when label num change
b=1   #txt position 2
cm=cm1
acc=acc1
feat=feat1
impt=impt1

df_cm = pd.DataFrame(cm, index=name1, columns=name2)
plt.figure()
sn.set(font_scale=1.5)
sn.heatmap(df_cm, annot=True,fmt="d",square=True,cbar=False,annot_kws={"size": size}) # font size
s01='accuracy={n:.3f}\nsensitivity={n11:.3f}\nprecision={n12:.3f}\nFeature Importance:\n{n1}={s1:.3f}\n{n2}={s2:.3f}\n{n3}={s3:.3f}\n{n4}={s4:.3f}'.\
    format(n=acc,n11=sen1,n12=pre1,\
        n1=feat[0],s1=impt[0],n2=feat[1],s2=impt[1],n3=feat[2],s3=impt[2],n4=feat[3],s4=impt[3])
plt.text(a,b,s01,fontsize=size)
tle=CVmethod+' CV NCS+SpO2 label '+ labelNum
plt.title(tle)
figure = plt.gcf()
figure.set_size_inches(w, h)

plt.savefig(Folder+r"/figPython/"+tle+".png", dpi=300)




cm=cm2
acc=acc2
feat=feat2
impt=impt2
df_cm = pd.DataFrame(cm, index=name1, columns=name2)
plt.figure()
sn.heatmap(df_cm, annot=True,fmt="d",square=True,cbar=False,annot_kws={"size": size}) # font size
sn.set(font_scale=1.5)
s01='accuracy={n:.3f}\nsensitivity={n11:.3f}\nprecision={n12:.3f}\nFeature Importance:\n{n1}={s1:.3f}\n{n2}={s2:.3f}\n{n3}={s3:.3f}\n{n4}={s4:.3f}'.\
    format(n=acc,n11=sen2,n12=pre2,\
        n1=feat[0],s1=impt[0],n2=feat[1],s2=impt[1],n3=feat[2],s3=impt[2],n4=feat[3],s4=impt[3])
plt.text(a,b,s01,fontsize=size)
tle=CVmethod+' CV NCS label '+ labelNum
plt.title(tle)
figure = plt.gcf()
figure.set_size_inches(w, h)
plt.savefig(Folder+r"/figPython/"+tle+".png", dpi=300)


cm=cm3
acc=acc3
feat=feat3
impt=impt3
df_cm = pd.DataFrame(cm, index=name1, columns=name2)
plt.figure()
sn.heatmap(df_cm, annot=True,fmt="d",square=True,cbar=False,annot_kws={"size": size}) # font size
sn.set(font_scale=1.5)
s01='accuracy={n:.3f}\nsensitivity={n11:.3f}\nprecision={n12:.3f}\nFeature Importance:\n{n1}={s1:.3f}\n{n2}={s2:.3f}\n{n3}={s3:.3f}\n{n4}={s4:.3f}'.\
    format(n=acc,n11=sen3,n12=pre3,\
        n1=feat[0],s1=impt[0],n2=feat[1],s2=impt[1],n3=feat[2],s3=impt[2],n4=feat[3],s4=impt[3])
plt.text(a,b,s01,fontsize=size)
tle=CVmethod+' CV PSG+SpO2 label '+ labelNum
plt.title(tle)
figure = plt.gcf()
figure.set_size_inches(w, h)
plt.savefig(Folder+r"/figPython/"+tle+".png", dpi=300)


cm=cm4
acc=acc4
feat=feat4
impt=impt4
df_cm = pd.DataFrame(cm, index=name1, columns=name2)
plt.figure()
sn.heatmap(df_cm, annot=True,fmt="d",square=True,cbar=False,annot_kws={"size": size}) # font size
sn.set(font_scale=1.5)
s01='accuracy={n:.3f}\nsensitivity={n11:.3f}\nprecision={n12:.3f}\nFeature Importance:\n{n1}={s1:.3f}\n{n2}={s2:.3f}\n{n3}={s3:.3f}\n{n4}={s4:.3f}'.\
    format(n=acc,n11=sen4,n12=pre4,\
        n1=feat[0],s1=impt[0],n2=feat[1],s2=impt[1],n3=feat[2],s3=impt[2],n4=feat[3],s4=impt[3])
plt.text(a,b,s01,fontsize=size)
tle=CVmethod+' CV PSG label '+ labelNum
plt.title(tle)
figure = plt.gcf()
figure.set_size_inches(w, h)
plt.savefig(Folder+r"/figPython/"+tle+".png", dpi=300)

In [ ]:



data=acc_all1
x = np.arange(len(data))  # the label locations
width = 0.5  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x+1, data, width, label='accuracy')


# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Accuracy')
ax.set_title('leave one CV NCS')
ax.set_xticks(x)
ax.set_xticklabels(x)
#ax.legend()




fig.tight_layout()


plt.savefig(r'C:\Users\zz587-admin\Documents\sleep center\result\OutputFeat_stage\featureAll\NCS_groupCV.png')

In [ ]:

# present case Num +label count 


data=caseNumNCS
x = np.arange(len(data))  # the label locations
width = 0.5  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x+1, data, width, label='accuracy')


# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Accuracy')
ax.set_title('leave one CV NCS')
ax.set_xticks(x)
ax.set_xticklabels(x)
#ax.legend()




fig.tight_layout()


#plt.savefig(r'C:\Users\zz587-admin\Documents\sleep center\result\OutputFeat_stage\featureAll\NCS_groupCV.png')
r=np.array([0.2,0])
featureNCS_del1,featureNCS_wo_del1,caseNumNCS_del1,labelNCS_3_del1,labelNCS_del1=\
    Del_outlier(featureNCS,featureNCS_wo,labelNCS,labelNCS_3,caseNumNCS,'forest','forest',r)
featureNCS_del2,featureNCS_wo_del2,caseNumNCS_del2,labelNCS_3_del2,labelNCS_del2=\
    Del_outlier(featureNCS,featureNCS_wo,labelNCS,labelNCS_3,caseNumNCS,'local','local',r)
featureNCS_del3,featureNCS_wo_del3,caseNumNCS_del3,labelNCS_3_del3,labelNCS_del3=\
    Del_outlier(featureNCS,featureNCS_wo,labelNCS,labelNCS_3,caseNumNCS,'SVM','SVM',r)
featureNCS_del4,featureNCS_wo_del4,caseNumNCS_del4,labelNCS_3_del4,labelNCS_del4=\
    Del_outlier(featureNCS,featureNCS_wo,labelNCS,labelNCS_3,caseNumNCS,'covar','covar',r)
